In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown

In [ ]:
#Helper Fuction
def import_from_gdrive(id_file, file_name):
  link = 'https://drive.google.com/uc?id=' + id_file
  output_file = '/content/' + file_name
  gdown.download(link, output_file, quiet=False)
  df = pd.read_csv(file_name)
  return df

In [ ]:
link_train_combined = '1RrJ73EqxTIME-tlSAU6Ob8n5_pxRbH_V'
link_test_combined = '1HkAMyEvIKKMXKuLPXMdbV_78vM00KXX0'
train = import_from_gdrive(link_train_combined,'train.csv')
test = import_from_gdrive(link_test_combined,'test.csv')

Downloading...
From: https://drive.google.com/uc?id=1RrJ73EqxTIME-tlSAU6Ob8n5_pxRbH_V
To: /content/train.csv
100%|██████████| 78.3M/78.3M [00:02<00:00, 29.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HkAMyEvIKKMXKuLPXMdbV_78vM00KXX0
To: /content/test.csv
100%|██████████| 25.0M/25.0M [00:00<00:00, 44.2MB/s]


In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398648 entries, 0 to 398647
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id_jalan          398648 non-null  int64  
 1   id_titik_mulai    398648 non-null  int64  
 2   id_titik_akhir    398648 non-null  int64  
 3   rerata_kecepatan  398648 non-null  float64
 4   kelompok          398648 non-null  object 
 5   titik1            398648 non-null  int64  
 6   titik2            398648 non-null  int64  
 7   maxspeed_mph      398648 non-null  int64  
 8   lat1              398648 non-null  float64
 9   long1             398648 non-null  float64
 10  lat2              398648 non-null  float64
 11  long2             398648 non-null  float64
 12  jarak_km          398648 non-null  float64
 13  tanggal           398648 non-null  int64  
 14  jam               398648 non-null  float64
 15  weekend           398648 non-null  object 
 16  libur             39

In [ ]:
import pandas as pd

num = train.copy()
num = num.drop(['kelompok','weekend','libur','aktivitas','cycleway','highway','sidewalk','surface'], axis= 1)

# Menghitung korelasi Pearson
correlation_matrix = num.corr()
rerata_kecepatan_correlation = correlation_matrix['rerata_kecepatan']
sorted_correlation = rerata_kecepatan_correlation.abs().sort_values(ascending=False)

print(sorted_correlation)

rerata_kecepatan    1.000000
jam                 0.301637
lat1                0.224003
lat2                0.223813
maxspeed_mph        0.205995
id_jalan            0.159702
titik2              0.105495
id_titik_akhir      0.099093
lanes               0.084585
jarak_km            0.072514
id_titik_mulai      0.070200
titik1              0.060273
long2               0.052464
long1               0.052220
tanggal             0.012503
Name: rerata_kecepatan, dtype: float64


In [ ]:
from scipy.stats import f_oneway

# Melakukan ANOVA untuk setiap fitur terhadap rerata_kecepatan
anova_results = {}
for feature in ['kelompok', 'weekend', 'libur', 'aktivitas', 'cycleway', 'highway', 'sidewalk', 'surface']:
    groups = train.groupby(feature)['rerata_kecepatan'].apply(list)
    anova_result = f_oneway(*groups)
    anova_results[feature] = anova_result

# Mengurutkan fitur-fitur berdasarkan nilai F (pada indeks 0 dalam hasil ANOVA)
sorted_anova_results = sorted(anova_results.items(), key=lambda x: x[1][0], reverse=True)

# Menampilkan hasil urutan fitur berdasarkan pengaruhnya
for feature, result in sorted_anova_results:
    print(f"Fitur: {feature}, Nilai F: {result[0]}, P-value: {result[1]}")

Fitur: aktivitas, Nilai F: 41392.92008407378, P-value: 0.0
Fitur: surface, Nilai F: 14711.59401947181, P-value: 0.0
Fitur: sidewalk, Nilai F: 7081.664299272504, P-value: 0.0
Fitur: cycleway, Nilai F: 6344.331397904684, P-value: 0.0
Fitur: highway, Nilai F: 5516.8492041962545, P-value: 0.0
Fitur: weekend, Nilai F: 3190.401839839407, P-value: 0.0
Fitur: kelompok, Nilai F: 475.91074005413327, P-value: 0.0
Fitur: libur, Nilai F: 344.6835355793602, P-value: 6.567900450630466e-77


In [ ]:
# num = num.drop(['libur','tgl','id_titik_mulai','id_titik_akhir'], axis= 1)

#Data Preparation

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
id_jalan_kelompok_train = train[['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface']]
encoded_train = encoder.fit_transform(id_jalan_kelompok_train)

id_jalan_kelompok_test =test[['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface']]
encoded_test = encoder.transform(id_jalan_kelompok_test)

In [ ]:
# Ubah hasil encoding menjadi DataFrame
encoded_train_df = pd.DataFrame(encoded_train, columns=['encoded_weekend','encoded_libur','encoded_aktivitas','encoded_cycleway','encoded_highway','encoded_sidewalk','encoded_surface'])
encoded_test_df = pd.DataFrame(encoded_test, columns=['encoded_weekend','encoded_libur','encoded_aktivitas','encoded_cycleway','encoded_highway','encoded_sidewalk','encoded_surface'])

# Gabungkan hasil encoding dengan data train dan test
train_clean = pd.concat([train, encoded_train_df], axis=1)
test_clean = pd.concat([test, encoded_test_df], axis=1)

In [ ]:
X_train = train_clean.drop(['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface'], axis= 1)
X_test = test_clean.drop(['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface'], axis= 1)
y_train = train_clean['rerata_kecepatan']

In [ ]:
ID = X_test['id']
X_test = X_test.drop(['id','kelompok'], axis=1)

In [ ]:
X_train = X_train.drop(['rerata_kecepatan','kelompok'], axis=1)

In [ ]:
X_train["maxspeed_mph"] = X_train["maxspeed_mph"] * 1.6
X_train.rename(columns={"maxspeed_mph": "maxspeed_kph"}, inplace=True)

X_test["maxspeed_mph"] = X_test["maxspeed_mph"] * 1.6
X_test.rename(columns={"maxspeed_mph": "maxspeed_kph"}, inplace=True)

In [ ]:
X_test = X_test.drop(['encoded_libur'], axis= 1)
X_train = X_train.drop(['encoded_libur'], axis= 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.drop(['titik1','titik2'], axis= 1)
X_test = X_test.drop(['titik1','titik2'], axis= 1)
X_val = X_val.drop(['titik1','titik2'], axis= 1)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318918 entries, 320768 to 121958
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id_jalan           318918 non-null  int64  
 1   id_titik_mulai     318918 non-null  int64  
 2   id_titik_akhir     318918 non-null  int64  
 3   maxspeed_kph       318918 non-null  float64
 4   lat1               318918 non-null  float64
 5   long1              318918 non-null  float64
 6   lat2               318918 non-null  float64
 7   long2              318918 non-null  float64
 8   jarak_km           318918 non-null  float64
 9   tanggal            318918 non-null  int64  
 10  jam                318918 non-null  float64
 11  lanes              318918 non-null  int64  
 12  encoded_weekend    318918 non-null  float64
 13  encoded_aktivitas  318918 non-null  float64
 14  encoded_cycleway   318918 non-null  float64
 15  encoded_highway    318918 non-null  float64
 1

In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127489 entries, 0 to 127488
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id_jalan           127489 non-null  int64  
 1   id_titik_mulai     127489 non-null  int64  
 2   id_titik_akhir     127489 non-null  int64  
 3   maxspeed_kph       127489 non-null  float64
 4   lat1               127489 non-null  float64
 5   long1              127489 non-null  float64
 6   lat2               127489 non-null  float64
 7   long2              127489 non-null  float64
 8   jarak_km           127489 non-null  float64
 9   tanggal            127489 non-null  int64  
 10  jam                127489 non-null  float64
 11  lanes              127489 non-null  int64  
 12  encoded_weekend    127489 non-null  float64
 13  encoded_aktivitas  127489 non-null  float64
 14  encoded_cycleway   127489 non-null  float64
 15  encoded_highway    127489 non-null  float64
 16  en

In [ ]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79730 entries, 143798 to 179896
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_jalan           79730 non-null  int64  
 1   id_titik_mulai     79730 non-null  int64  
 2   id_titik_akhir     79730 non-null  int64  
 3   maxspeed_kph       79730 non-null  float64
 4   lat1               79730 non-null  float64
 5   long1              79730 non-null  float64
 6   lat2               79730 non-null  float64
 7   long2              79730 non-null  float64
 8   jarak_km           79730 non-null  float64
 9   tanggal            79730 non-null  int64  
 10  jam                79730 non-null  float64
 11  lanes              79730 non-null  int64  
 12  encoded_weekend    79730 non-null  float64
 13  encoded_aktivitas  79730 non-null  float64
 14  encoded_cycleway   79730 non-null  float64
 15  encoded_highway    79730 non-null  float64
 16  encoded_sidewalk

#LBGM

In [ ]:
import lightgbm as lgb

In [ ]:
# output.describe()

,id,rerata_kecepatan
count,127489.000000,127489.000000
mean,63744.000000,35.486497
std,36803.048572,7.592135
min,0.000000,5.133412
25%,31872.000000,31.434407
50%,63744.000000,36.220509
75%,95616.000000,40.695002
max,127488.000000,54.227241


In [ ]:
params = {
    "application": "regression",
    "boosting": "gbdt",
    "metric": "rmse",
    "num_leaves": 135,
    "max_depth": 11,
    "learning_rate": 0.052586623405764234,
    "bagging_fraction": 0.6644887700856914,
    "feature_fraction": 0.8230021324729027,
    "min_split_gain": 0.0025435434125472204,
    "min_child_samples": 182,
    "min_child_weight": 0.023336215175678358,
    "lambda_l2": 0.0372533252920578,
    "verbosity": -1,
    "data_random_seed": 17,
}

# Additional parameters:
early_stop = 500
verbose_eval = 100
num_rounds = 10000

d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_val, label=y_val)
valid_sets = [d_train, d_valid]

lgbm_model = lgb.train(params, train_set=d_train,num_boost_round=num_rounds, valid_sets=valid_sets, verbose_eval=verbose_eval,early_stopping_rounds=early_stop,)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 500 rounds
[100]	training's rmse: 3.82799	valid_1's rmse: 3.88117
[200]	training's rmse: 3.4794	valid_1's rmse: 3.54352
[300]	training's rmse: 3.30331	valid_1's rmse: 3.37828
[400]	training's rmse: 3.19425	valid_1's rmse: 3.28208
[500]	training's rmse: 3.1216	valid_1's rmse: 3.22047
[600]	training's rmse: 3.07087	valid_1's rmse: 3.1814
[700]	training's rmse: 3.03065	valid_1's rmse: 3.15264
[800]	training's rmse: 2.99863	valid_1's rmse: 3.13141
[900]	training's rmse: 2.97077	valid_1's rmse: 3.114
[1000]	training's rmse: 2.94105	valid_1's rmse: 3.09487
[1100]	training's rmse: 2.91847	valid_1's rmse: 3.08137
[1200]	training's rmse: 2.89662	valid_1's rmse: 3.06937
[1300]	training's rmse: 2.87765	valid_1's rmse: 3.05832
[1400]	training's rmse: 2.86001	valid_1's rmse: 3.04889
[1500]	training's rmse: 2.84187	valid_1's rmse: 3.03821
[1600]	training's rmse: 2.82423	valid_1's rmse: 3.02898
[1700]	training's rmse: 2.80967	valid_1's rmse: 3.02195


In [ ]:
# Prediksi dengan model yang telah dilatih
y_pred = lgbm_model.predict(X_test, num_iteration=lgbm_model.best_iteration)
y_pred

array([42.92654191, 42.04379846, 36.66410097, ..., 41.23916141,
       35.54978383, 44.68696809])

In [ ]:
output = pd.DataFrame({'id': ID,
                       'rerata_kecepatan': y_pred})

In [ ]:
output.describe()

,id,rerata_kecepatan
count,127489.000000,127489.000000
mean,63744.000000,35.455081
std,36803.048572,7.862090
min,0.000000,2.250829
25%,31872.000000,31.280944
50%,63744.000000,36.255393
75%,95616.000000,40.851802
max,127488.000000,56.354971


In [ ]:
output.to_csv('LGBMselection.csv', index=False)